# A training example in Pytorch
## Introduction
### Task
В этом ноутбуке мы научим нейронную сеть выполнять простую задачу. Это будет задача классификации. Классификация в основном означает нахождение *decision boundary* в пространстве вещественных чисел. В целях понятности мы будем работать с 2D-примером: *decision boundary* будет круг. Точнее, единичный круг на плоскости.

![](unitycircle.png)
### Sampling
Будем генерировать точки $(x_1,x_2)$ для классификации, и их класс $y$. Настоящая разделяющая функция будет $y=1_{x_1^2+x_2^2<1}$.

Чтобы иметь сбалансированный набор данных с примерно одинаковым количеством точек в каждом классе, мы будем производить выборку равномерно по полярным координатам, в пределах круга с центром 0 и радиусом 2.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

# если гпу у вас несколько укажите номер гпу, которую хотите использовать
# номер свободной можно посмотреть с помощью nvidia-smi в терминале
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
def sample_points(n):
    # returns (X,Y), where X of shape (n,2) is the numpy array of points and Y is the (n) array of classes
    
    radius = np.random.uniform(low=0,high=2,size=n).reshape(-1,1) # uniform radius between 0 and 2
    angle = np.random.uniform(low=0,high=2*np.pi,size=n).reshape(-1,1) # uniform angle
    x1 = radius*np.cos(angle)
    x2=radius*np.sin(angle)
    y = (radius<1).astype(int).reshape(-1)
    x = np.concatenate([x1,x2],axis=1)
    return x,y

In [ ]:
# Generate the data
trainx,trainy = sample_points(10000)
valx,valy = sample_points(500)
testx,testy = sample_points(500)

print(trainx.shape,trainy.shape)

Наша модель будет многослойным перцептроном с одним скрытым слоем и выходом размера 2, так как у нас есть два класса. Поскольку это задача бинарной классификации, мы могли бы также использовать только один выход и нулевой порог, но мы будем использовать два, чтобы проиллюстрировать использование кросс-энтропийной функции потерь в pytorch (на следующей неделе вы увидите, как использовать BinaryCrossEntropy для такой задачи) ,

Многослойный перцептрон не может выучить круговую границу, но может представлять границу - полигон, число сторон которого равно числу нейронов на скрытом слое. Например, с 6 скрытыми нейронами модель может вычислить гексагональную границу, которая приближается к единичному кругу, например:
![](hexagon.png)

Обратите внимание, что оптимальная точность (accuracy), ожидаемая от гексагона, аппроксимирующего единичный круг, составляет около **xxxx**. Конечно, обученная модель не будет вычислять фактический гексагон из-за активации, которая не является порогом, и свободы весов конечного слоя. На самом деле мы можем ожидать *лучшую* точность, чем эта.

In [ ]:
def generate_single_hidden_MLP(n_hidden_neurons):
    return nn.Sequential(nn.Linear(2,n_hidden_neurons),nn.ReLU(),nn.Linear(n_hidden_neurons,2))
model1 = generate_single_hidden_MLP(6)

Чтобы обучить нашу модель, нам нужно будет кормить ее тензорами. Давайте преобразуем наши сгенерированные массивы NumPy

In [ ]:
trainx = torch.from_numpy(trainx).float()
valx = torch.from_numpy(valx).float()
testx = torch.from_numpy(testx).float()
trainy = torch.from_numpy(trainy)
valy = torch.from_numpy(valy)
testy = torch.from_numpy(testy)
print(trainx.type(),trainy.type())

Теперь мы определимся с нашей тренировочной рутиной. Возникает вопрос о том, выполнять ли тренировку на CPU or GPU. Лучше всего использовать flag переменную, которую вы будете устанавливать, когда будете выполнять обучение.

In [ ]:
def training_routine(net,dataset,n_iters,gpu):
    # organize the data
    train_data,train_labels,val_data,val_labels = dataset
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(),lr=0.01)
    
    # use the flag
    if gpu:
        train_data,train_labels = train_data.cuda(),train_labels.cuda()
        val_data,val_labels = val_data.cuda(),val_labels.cuda()
        net = net.cuda() # the network parameters also need to be on the gpu !
        print("Using GPU")
    else:
        print("Using CPU")
    for i in range(n_iters):
        # forward pass
        train_output = net(train_data)
        train_loss = criterion(train_output,train_labels)
        # backward pass and optimization
        train_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # Once every 100 iterations, print statistics
        if i%100==0:
            print("At iteration",i)
            # compute the accuracy of the prediction
            train_prediction = train_output.cpu().detach().argmax(dim=1)
            train_accuracy = (train_prediction.numpy()==train_labels.numpy()).mean() 
            # Now for the validation set
            val_output = net(val_data)
            val_loss = criterion(val_output,val_labels)
            # compute the accuracy of the prediction
            val_prediction = val_output.cpu().detach().argmax(dim=1)
            val_accuracy = (val_prediction.numpy()==val_labels.numpy()).mean() 
            print("Training loss :",train_loss.cpu().detach().numpy())
            print("Training accuracy :",train_accuracy)
            print("Validation loss :",val_loss.cpu().detach().numpy())
            print("Validation accuracy :",val_accuracy)
    
    net = net.cpu()

In [ ]:
dataset = trainx,trainy,valx,valy

In [ ]:
gpu = False
gpu = gpu and torch.cuda.is_available() # to know if you actually can use the GPU

training_routine(model1,dataset,10000,gpu)

In [ ]:
# Let's try with 3 hidden neurons.
model2 = generate_single_hidden_MLP(3) 
training_routine(model2,dataset,10000,gpu)

In [ ]:
out = model2(testx).argmax(dim=1).detach().numpy()
green = testx.numpy()[np.where(out==1)]
red = testx.numpy()[np.where(out==0)]
print(green.shape,red.shape)

In [ ]:
def print_model(model,datapoints):
    out = model(datapoints).argmax(dim=1).detach().numpy()
    green = datapoints.numpy()[np.where(out==1)]
    red = datapoints.numpy()[np.where(out==0)]

    circle1 = plt.Circle((0, 0), 1, color='y')
    circle2 = plt.Circle((0, 0), 1, color='b',fill=False)

    fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot
    # (or if you have an existing figure)
    # fig = plt.gcf()
    # ax = fig.gca()
    plt.xlim((-2,2))
    plt.ylim((-2,2))

    pos_values = plt.scatter(x=green[:,0],y=green[:,1], color='g',)
    neg_values = plt.scatter(x=red[:,0],y=red[:,1], color='r',)

    ax.add_artist(circle1)
    ax.add_artist(circle2)
    ax.add_artist(pos_values)
    ax.add_artist(neg_values)

In [ ]:
print_model(model1,testx)

In [ ]:
print_model(model2,testx)

In [ ]:
model3 = generate_single_hidden_MLP(2) 
training_routine(model3,dataset,10000,gpu)

In [ ]:
print_model(model3,testx)